---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:
    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import re
    df = pd.read_table('university_towns.txt',header = None)
    lst = []
    state = str(df.iloc[0].values)[2:-8]
    for i in range(1,len(df)):
        if '[edit]' not in str(df.iloc[i].values):
            lst.append([state,str(df.iloc[i].values)[2:-1]])
        else:
            state = str(df.iloc[i].values)[2:-8]
    df1 = pd.DataFrame(lst,columns = ['State','RegionName'])
    letter = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

    for i in range(0,len(df1['RegionName'])):
        head, sep, tail = df1['RegionName'].iloc[i].partition(' (')
        if head[0] not in letter:
            head = head[1:]
        if head[-1] == '\'':
            head = head[:-1]
        df1['RegionName'].iloc[i] = head
    

    return df1
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # import pandas as pd
    # import numpy as np
    # import scipy.stat
    towns = pd.read_table("university_towns.txt",header=None,names=["RegionName"])
    4
    current_state = ""
    def get_state(cell):
        if cell.endswith("[edit]"):
            global current_state
            current_state = cell[:-6]
            return cell[:-6]
        else:
            return current_state

    towns["State"] = towns["RegionName"].map(get_state)
    towns = towns[~towns["RegionName"].str.endswith("[edit]")]
    towns["RegionName"] = towns["RegionName"].map(lambda x:x.split("(")[0].strip())
    towns = towns.reindex(columns=["State","RegionName"]).reset_index(drop=True)
    towns.State = towns.State.map(dict(zip(states.values(),states.keys())))       # using abbreviation for state 
    
    return towns

In [5]:
get_list_of_university_towns()

,State,RegionName
0,AL,Auburn
1,AL,Florence
2,AL,Jacksonville
3,AL,Livingston
4,AL,Montevallo
5,AL,Troy
6,AL,Tuscaloosa
7,AL,Tuskegee
8,AK,Fairbanks
9,AZ,Flagstaff


In [6]:
gdp = (pd.read_excel("gdplev.xls",header=1) #reading gdplev file
                 .drop(["Annual","Unnamed: 1","Unnamed: 2","Unnamed: 3","Unnamed: 5","Unnamed: 7"],axis=1) #
                 .iloc[217:] #droping column that are unncessary
                 .rename(columns={"Unnamed: 6":"GDP"}) #
                 .reset_index(drop=True) #
        )
gdp.GDP = gdp.GDP.astype(np.float64)

old = None
def do_math(cell):
    global old
    if old == None: old = cell
    delta = cell - old
    old = cell
    return delta

before = None
def again_math(cell):
    global before
    if before == None: before = 0
    before_cell = before
    before = cell
    if cell < 0 and before_cell < 0:
        return "DES"
    elif cell > 0 and before_cell > 0:
        return "INS"
    else:
        return np.nan


gdp["Delta"] = gdp["GDP"].map(do_math)
gdp["Type"] = gdp["Delta"].map(again_math)

gdp.head()

,Quarterly,GDP,Delta,Type
0,2000q1,12359.1,0.0,NaN
1,2000q2,12592.5,233.4,NaN
2,2000q3,12607.7,15.2,INS
3,2000q4,12679.3,71.6,INS
4,2001q1,12643.3,-36.0,NaN


In [7]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    start = []
    for i in gdp[gdp.Type == "DES"].index:
        if gdp.iloc[i-1].Type != "DES" and gdp.iloc[i-1].Type != "INS":
            start.append(i-1)
    res = gdp.iloc[start][["Quarterly"]].values[0,0]
    return res
get_recession_start()

   

'2008q3'

In [8]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    quart = get_recession_start()
    index = gdp[gdp.Quarterly == quart].index[0]
    while True:
        if gdp.iloc[index].Delta > 0 and  gdp.iloc[index+1].Delta > 0:
            ends = index + 1
            break
        else:
            index += 1
            if index > 10000000: return None
    res_end = gdp.iloc[ends]["Quarterly"]
    return res_end

get_recession_end()

'2009q4'

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    start = get_recession_start()
    s_index = gdp[gdp.Quarterly == start].index[0]
    ends = get_recession_end()
    e_index = gdp[gdp.Quarterly == ends].index[0]

    index = s_index
    mins = ""
    while True:
        #print(index,gdp.iloc[index+1].GDP)
        if gdp.iloc[index+1].GDP < gdp.iloc[index].GDP:
            mins = gdp.iloc[index+1].Quarterly
            mins2 = index + 1
        else:
            pass
        index += 1
        if index >= e_index: break

    return mins
get_recession_bottom()

'2009q2'

In [12]:
def get_recession_bottom():
    """
    Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3
    """
    df = pd.read_excel('https://storage.googleapis.com/um_ds_intro/gdplev.xls', skiprows=220, header=None)
    df = df.iloc[:, [4, 6]]
    df.columns = ['Quarterly', 'GDP in billions of chained 2009 dollars']
    df = df.set_index('Quarterly')
    recession_starts = get_recession_start()
    recession_ends = get_recession_end()
    df_recession = df.loc[recession_starts:recession_ends, :]
    ans = df_recession.idxmin().values[0]
    return ans
get_recession_bottom()

'2009q2'

In [13]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    import pandas as pd
    df_gdp = pd.read_excel('gdplev.xls', skiprows=220, parse_cols='E,G', header=None)
    df_gdp.columns = ['quarter', 'GDP']

    for i in range(4, len(df_gdp)):
        if (df_gdp.loc[i-4, 'GDP'] > df_gdp.loc[i-3, 'GDP']) \
                and (df_gdp.loc[i-3, 'GDP'] > df_gdp.loc[i-2, 'GDP']) \
                and (df_gdp.loc[i-2, 'GDP'] < df_gdp.loc[i-1, 'GDP']) \
                and (df_gdp.loc[i-1, 'GDP'] < df_gdp.loc[i, 'GDP']):
            result = df_gdp.loc[i-2, 'quarter']

    return result
get_recession_bottom()

'2009q2'

In [27]:
def get_gdp_df():
    '''this function reads the gdp data and returns a dataframe with only the required columns'''
    gdplev = pd.ExcelFile('gdplev.xls')
    gdplev = gdplev.parse("Sheet1", skiprows=219)
    gdplev = gdplev[['1999q4', 9926.1]]
    gdplev.columns = ['quarter','gdp']
    return gdplev
get_gdp_df()

###############

,quarter,gdp
0,2000q1,10031.0
1,2000q2,10278.3
2,2000q3,10357.4
3,2000q4,10472.3
4,2001q1,10508.1
5,2001q2,10638.4
6,2001q3,10639.5
7,2001q4,10701.3
8,2002q1,10834.4
9,2002q2,10934.8


In [32]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].

    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.

    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house = pd.read_csv("City_Zhvi_AllHomes.csv")
    ihouse = house.loc[:,"2000-01":"2016-08"]#提取2000年后纯时间序列进行分析
    ihouse.columns = pd.to_datetime(ihouse.columns).to_period(freq="M")#调整为Period-Month模式显示
    ghouse = ihouse.groupby(ihouse.columns.asfreq("Q"),axis=1).sum()#按照column的Period-Q季度模式进行分组，合并值
    house = (pd.merge(house.loc[:,"RegionID":"SizeRank"],ghouse,left_index=True,right_index=True,how="inner")#合并数据
    .set_index(["State","RegionName"]).iloc[:,4:71])#整理索引排序
    return house
convert_housing_data_to_quarters().size

718910

In [37]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''

    import pandas as pd
    import numpy as np
    df = pd.read_csv('City_Zhvi_AllHomes.csv')

    # Create columns to keep
    cols_to_keep = ['RegionID', 'RegionName', 'State']
    for i in range(2000, 2017):
        for j in range(1, 13):
            if j <= 9:
                if i == 2016 and j == 9:
                    pass
                else:
                    month_str = '0' + str(j)
            else:
                if i == 2016:
                    pass
                else:
                    month_str = str(j)
            cols_to_keep.append(str(i) + '-' + month_str)
    df = df[cols_to_keep]

    # Convert two letter state abbreviations to state names
    df['State'] = df['State'].replace(states)

    def convert_to_qtr(ym):
        year, month = ym.split('-')
        if month == '01' or month == '02' or month == '03':
            result = year + 'q1'
        elif month == '04' or month == '05' or month == '06':
            result = year + 'q2'
        elif month == '07' or month == '08' or month == '09':
            result = year + 'q3'
        else:
            result = year + 'q4'
        return result

    # Stack the columns of GDP values and add a quarter column
    df_compiled = df.copy().set_index(['State', 'RegionName', 'RegionID']).stack(dropna=False)
    df_compiled = df_compiled.reset_index().rename(columns={'level_3': 'year_month', 0: 'gdp'})
    df_compiled.drop_duplicates(inplace=True)
    df_compiled['quarter'] = df_compiled['year_month'].apply(convert_to_qtr)
    df_compiled = df_compiled.drop('year_month', axis=1)
    result = df_compiled.pivot_table(values='gdp', index=['State', 'RegionName', 'RegionID'], columns='quarter',
                                     aggfunc=np.mean)
    result = result.reset_index()
    result = result.drop('RegionID', axis=1)
    # del result.index.name
    result = result.set_index(['State', 'RegionName'])
    return result
convert_housing_data_to_quarters()

quarter                           2000q1         2000q2         2000q3  \
State     RegionName                                                     
Alabama   Adamsville        69033.333333   69166.666667   69800.000000   
          Alabaster        122133.333333  123066.666667  123166.666667   
          Albertville       73966.666667   72600.000000   72833.333333   
          Arab              83766.666667   81566.666667   81333.333333   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       81700.000000   78533.333333   79133.333333   
          Bayou La Batre    44066.666667   44500.000000   44266.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54033.333333   54400.000000   54966.666667   
          Boaz              70866.666667   70266.666667   70300.000000   
          Brent             92933.333333   94333.333333   96166.666667   
          Brighton                   NaN            NaN            NaN   
          Brookwood         92566.666667   95100.000000   98866.666667   
          Buhl              90800.000000   94600.000000   96500.000000   
          Calera           108933.333333  110366.666667  108000.000000   
          Center Point      80966.666667   81233.333333   81500.000000   
          Centreville       95300.000000   96566.666667   98000.000000   
          Chalkville        94100.000000   94433.333333   94433.333333   
          Chancellor                 NaN            NaN            NaN   
          Chelsea          162066.666667  167033.333333  166900.000000   
          Chickasaw         51200.000000   53666.666667   54933.333333   
          Chunchula         80266.666667   81766.666667   82200.000000   
          Citronelle        64833.333333   66633.333333   68066.666667   
          Clay             120900.000000  122266.666667  123966.666667   
          Coden             62600.000000   64800.000000   66866.666667   
          Coker            118100.000000  120766.666667  118166.666667   
          Concord           78600.000000   78700.000000   80133.333333   
          Cottondale       100833.333333  102633.333333  104766.666667   
...                                  ...            ...            ...   
Wisconsin Vernon           183200.000000  178200.000000  174300.000000   
          Vienna           178033.333333  181533.333333  182433.333333   
          Vinland          119800.000000  126766.666667  134933.333333   
          Wales                      NaN            NaN            NaN   
          Waterford        121200.000000  119433.333333  120200.000000   
          Waukesha         141266.666667  138433.333333  136733.333333   
          Waunakee         187400.000000  191433.333333  192666.666667   
          Waupun            84000.000000   84000.000000   84400.000000   
          Wausau            69566.666667   69466.666667   71033.333333   
          Wayne                      NaN            NaN            NaN   
          West Allis        80933.333333   81933.333333   84066.666667   
          Weston            80166.666667   82700.000000   84166.666667   
          Wheatland        129800.000000  132133.333333  134933.333333   
          Whitelaw          96033.333333  101433.333333  108033.333333   
          Williams Bay     122900.000000  115300.000000  110766.666667   
          Wilson                     NaN            NaN            NaN   
          Wilson           129033.333333  129366.666667  132433.333333   
          Wind Lake        124366.666667  123666.666667  124133.333333   
          Wind Point       149233.333333  145266.666667  140866.666667   
          Wisconsin Dells  100000.000000  103400.000000  105000.000000   
          Wolf River        95166.666667   99333.333333  103800.000000   
    

In [11]:
from scipy import stats
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 

    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    start = pd.Period(get_recession_start())
    bottom = pd.Period(get_recession_bottom())
    house = convert_housing_data_to_quarters().loc[:,[start,bottom]]
    house.columns = ["Start","Bottom"]
    house["Ratio"] = house.Start / house.Bottom #NAN不用处理，反正数据不使用
    house = house.dropna(axis=0,how="any")
    collage = get_list_of_university_towns().set_index(["State","RegionName"])
    collage["isUnv"] = "Yes"
    res = pd.merge(house,collage,how="left",left_index=True,right_index=True)
    res.isUnv = res.isUnv.fillna("No")

    res_u = res[res.isUnv == "Yes"].Ratio
    res_n = res[res.isUnv == "No"].Ratio
    #print(res_n)
    _,p = stats.ttest_ind(res_u,res_n)
    different = (True if p < 0.01 else False)
    better = ("university town" if np.nanmean(res_u) < np.nanmean(res_n) else "non-university town")
    return different, p, better
run_ttest()

(True, 0.005932482020709121, 'university town')